In [27]:
# go wide screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [28]:
from astropy.io import fits, ascii
import spectrum

#%matplotlib inline
# To prevent automatic figure display when execution of the cell ends
%config InlineBackend.close_figures=True 

import matplotlib.pyplot as plt
import numpy as np

from IPython.html import widgets
from ipywidgets import Layout
from IPython.display import display,clear_output

from scipy.interpolate import interp1d

from astropy.wcs import WCS

from collections import OrderedDict


In [29]:
def register_ds9staircase():
    # register color map
    from matplotlib.cm import register_cmap, cmap_d

    colors = []
    for ii in range(1,6):
        kk = ii/5.
        colors.append( (kk*.3,kk*.3,kk*1)  )

    for ii in range(1,6):
        kk = ii/5.
        colors.append( (kk*.3,kk*1,kk*.3)  )
    for ii in range(1,6):
        kk = ii/5.
        colors.append( (kk*1,kk*.3,kk*.3)  )
    colors = np.array(colors)
    xx = np.arange(len(colors), dtype=float)
    xx = xx/xx.max()

    ds9staircase = {'red': lambda v : np.interp(v, xx, colors[:,0]),
               'green': lambda v : np.interp(v, xx, colors[:,1]),
               'blue': lambda v : np.interp(v, xx, colors[:,2])}


    # Register all other colormaps
    register_cmap('ds9staircase', data=ds9staircase)

register_ds9staircase()

In [30]:
cat = ascii.read("../data/mmsf2outcube_COSMOSC_allifu.cat")
#cat.sort(("claifu","id"))

#ii = (cat["class2"] == "halo") +  (cat["class2"] == "filament") + ((cat["class2"] == "pointsource") * cat["use_stack"] )
#ii *= cat["good"] # Good is too strong a selector, these were only the few posterbookexamples I selected in a non-systematic fashion
ii = (cat["class2"] == "pointsource")
ii *= cat["split_good"] == "True"
#ii *= cat["use_stack"]


#ii = cat["class2"] == "agn"
cat = cat[ii]
print("{} sources.".format(sum(ii)))

95 sources.


In [31]:
field = "COSMOSC"
cache = OrderedDict()

In [32]:
ftcal = "../specphot/mean_cal_COSMOSC.txt"
tcal = ascii.read(ftcal, format="fixed_width")
cal_interp = interp1d(tcal["wl[A]"], tcal["cal[erg/s/cm^2/A/cnt]"], kind='cubic', bounds_error=False)

In [33]:
# load rxay catalog
from astropy.table import Table
cls = Table.read("../civano+2016_cls.xml")

In [34]:
colormap = plt.get_cmap('ds9staircase')

from astropy.table import Table
cls = Table.read("../civano+2016_cls.xml")


def show_slice(ax, s, r, cal_interp, width = 50./3600., vmin=0., vmax=.5, cls=cls):
    ra,dec = r["ra_com"], r["dec_com"]

    hdu = s.hdu
    w = WCS(hdu.header)
    w = w.dropaxis(2)
    colormap = plt.get_cmap('ds9staircase')
    sl = hdu.data[int(np.round(r["z_com"]))]
    cos_term = np.cos(np.deg2rad(dec))

    sl[sl == 0] = np.nan
    colormap.set_bad(color='grey')

    ax.imshow(sl, vmin=vmin, vmax=vmax, origin='lower', interpolation='nearest', cmap=colormap)
    x0,y0 = w.wcs_world2pix(ra - width/2./cos_term ,dec-width/2.,0)
    x1,y1 = w.wcs_world2pix(ra + width/2./cos_term, dec+width/2.,0)
    x,y = w.wcs_world2pix(ra,dec,0)
    ax.plot([x],[y],'x',c='white')
    
    
    ax.set_xlim([x1,x0])
    ax.set_ylim([y0,y1])
    ax.set_xlabel("RA")
    ax.set_ylabel("Dec")

    if type(cls) != type(None):
        cls_ra  = np.array( cls["RAJ2000"], dtype=float) 
        cls_dec = np.array( cls["DEJ2000"], dtype=float) 
        # plot xray detections
        ii  = cls_ra  >= ra - width/2.
        ii *= cls_ra  <= ra + width/2.
        ii *= cls_dec >= dec - width/2.
        ii *= cls_dec <= dec + width/2.
        x,y = w.wcs_world2pix(cls_ra[ii],cls_dec[ii],0)
        ax.plot(x,y,'D',c='green')

    return sl

In [35]:
np.unique( cat["class2"] )

pointsource


In [36]:
len(cat)

95

In [37]:
cat

ifu,id,N,counts,flux,ra_com,dec_com,ddec,dra,x_com,y_com,z_com,dx,dy,dz,sqrt(ev1),sqrt(ev2),size2d,x_ext,y_ext,z_ext,wl_com,dwl,xmin,xmax,ymin,ymax,zmin,zmax,class,manualx,manualy,manualz,class2,comment2,use_stack,split_good,showcase,xray_counterpart
,,px,counts,erg / (cm2 s),RA[J2000],Dec[J2000],arcsec,arcsec,px,px,px,px,px,px,px,px,px,px,px,px,A,A,px,px,px,px,px,px,,,,,,,,,,
str3,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,int64,int64,int64,int64,int64,int64,str14,float64,float64,float64,str11,str135,bool,object,bool,bool
021,10001,773,2.2536e+02,9.0865e-17,150.330209,2.259786,1.5,1.4,89.34,92.92,62.10,2.91,2.73,3.53,3.32,2.23,4.00,17,15,17,3616.08,33.76,81,98,86,101,52,69,N/A,91.0,94.0,62.0,pointsource,LAE,False,True,False,False
021,5006,193,9.6403e+01,1.9428e-17,150.329382,2.251960,1.0,0.7,95.29,36.57,245.19,2.08,1.45,1.29,2.10,1.42,2.54,8,6,5,3979.66,9.93,91,99,34,40,243,248,N/A,96.0,36.0,244.0,pointsource,N/A,True,True,False,False
021,5007,200,1.0009e+02,1.7523e-17,150.334247,2.254039,0.9,0.7,60.29,51.53,284.73,1.77,1.40,1.88,1.82,1.34,2.26,8,5,8,4058.19,15.89,56,64,49,54,281,289,N/A,60.0,51.0,284.0,pointsource,N/A,False,True,False,False
021,5008,150,7.3210e+01,1.2414e-17,150.336250,2.261962,0.7,0.9,45.88,108.58,299.92,1.37,1.88,1.55,1.35,1.90,2.33,6,8,7,4088.34,13.90,43,49,105,113,295,302,N/A,46.0,108.0,300.0,pointsource,N/A,False,True,False,False
021,5013,510,3.0163e+02,4.6466e-17,150.327640,2.254576,1.0,1.0,107.82,55.40,396.29,2.10,1.91,4.55,2.19,1.80,2.84,11,9,21,4279.73,41.70,103,114,51,60,386,407,N/A,107.0,56.0,398.0,pointsource,N/A,False,True,False,False
022,10001,515,1.6196e+02,5.3227e-17,150.314579,2.278018,1.1,1.4,95.64,54.10,102.28,2.21,2.82,3.80,1.91,3.03,3.58,11,15,17,3695.87,33.76,90,101,44,59,96,113,N/A,95.0,55.0,101.0,pointsource,tadpole structure.,False,True,False,False
022,10002,727,2.7101e+02,8.3691e-17,150.318529,2.277924,0.8,1.6,67.22,53.42,115.35,1.63,3.23,3.58,1.41,3.34,3.62,8,16,15,3721.82,29.79,63,71,44,60,107,122,N/A,68.0,54.0,118.0,pointsource,N/A,False,True,False,False
022,10006,407,1.6791e+02,3.3398e-17,150.317517,2.276015,1.0,1.0,74.50,39.68,248.58,1.97,2.01,2.85,1.87,2.10,2.82,9,10,11,3986.39,21.84,70,79,34,44,242,253,N/A,74.0,39.0,250.0,pointsource,N/A,True,True,False,False


In [38]:
rbuttons = {}
cboxes_xray = {}

for i, record in enumerate( cat[:][:100] ):
    try:
        ifu = record["ifu"]

        fo = "../data/sf2outcube_{}_{}.fits.gz".format(field, ifu)
        fA = "../data/splits/splitA/sf2outcube_{}_{}.fits.gz".format(field, ifu)
        fB = "../data/splits/splitB/sf2outcube_{}_{}.fits.gz".format(field, ifu)

        if not fo in cache:
            cache[fo] = spectrum.readSpectrum(fo)
        if not fA in cache:
            cache[fA] = spectrum.readSpectrum(fA)
        if not fB in cache:
            cache[fB] = spectrum.readSpectrum(fB)

        so = cache[fo]
        sA = cache[fA]
        sB = cache[fB]

        f = plt.figure(figsize = [17,5])
        hdu = so.hdu
        w = WCS(hdu.header)
        w = w.dropaxis(2)

        ax1 = plt.subplot(131, projection=w)
        sl = show_slice(ax1, so, record, cal_interp, vmin=0., vmax=.5)

        ax2 = plt.subplot(132, projection=w)
        sl = show_slice(ax2, sA, record, cal_interp, vmin=0., vmax=.79, cls=None)

        ax3 = plt.subplot(133, projection=w)
        s3 = show_slice(ax3, sB, record, cal_interp, vmin=0., vmax=.79, cls=None)


        cbox_xray = widgets.Checkbox(
            description='xray counterpart',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='arrow'
        )
        
        cboxes_xray[(record["ifu"],record["id"])] = cbox_xray


        rbutton_good = widgets.RadioButtons(
            options=['unclear', 'yes', 'no'],
            description='Detected in split exposure:',
            disabled=False
        )

        rbuttons[(record["ifu"],record["id"])] = rbutton_good

        canvas  = widgets.Output(layout=Layout(border='light'))
        info = widgets.HTML(
            value="Hello <b>World</b>"
        )


        hb = widgets.HBox([info,canvas, rbutton_good, cbox_xray], layout=Layout(width = '1500px', positioning="bottom"))

        display(hb)

        with canvas:
            clear_output(wait=True)
            display(f)

        cbox_xray.value = bool( record["xray_counterpart"] )
        
        if "split_good" in cat.colnames:
            if record["split_good"] == "True":
                rbutton_good.value = 'yes'
            elif record["split_good"] == "False":
                rbutton_good.value = 'no'
            else:
                rbutton_good.value = 'unclear'



        ncols = 5
        s = ""
        s = "<style>"
        s = "table, th, td {"
        s = "  border: 1px solid black;"
        s = "  border-collapse: collapse;"
        s = "}"
        s = "th, td {"
        s = "  padding: 10px;"
        s = "}"
        s = "</style>"
        tablerows = []
        sr = ""
        z_lya = record["wl_com"] /1216. - 1.
        z_oii = record["wl_com"] /3727. - 1.

        s += "<b>{} ifu = {}, id = {}</b><br>\n".format(i, record["ifu"], record["id"] )
        s += "<b>class2 = {} </b><br>\n".format( record["class2"]  )
        s += "<b>z_LyA = {:.2f}</b><br>\n".format(z_lya)
        s += "<b>z_OII = {:.2f}</b><br>\n".format(z_oii)

        s += "<table style='width:100%'>\n"

        for i,n in enumerate(record.colnames[2:]):
            sr += "<td><b>{:10s}:</b> ".format(n)
            sr += str(record[n]) + " "
            if record.columns[n].unit != "":
                sr += str(record.columns[n].unit) + "&nbsp;&nbsp;&nbsp;</td>\n"
            else:
                sr += "</td> \n"
            if i % ncols == 0.:
                tablerows.append(sr)
                sr = ""
        s += "</tr>\n"
        s += "</tr>\n<tr>".join(tablerows)       
        s += "</tr>\n"
        s += "</table>\n"


        #info.value = s
        info.value = "{}<br>{}<br>{}<br>{}".format(record["ifu"], record["id"], record["wl_com"], record["class2"])
        
    except:
        print("Error in ifu {} ,id {}".format(record["ifu"], record["id"]))
    

/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


...

/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [ ]:
from astropy.table import Column
cat = ascii.read("../data/mmsf2outcube_COSMOSC_allifu.cat")
if not "split_good" in cat.colnames:
    cat.add_column(Column([None]*len(cat), name="split_good", description="Source still detected in split cubes.", dtype="S4"))


print("Split detect")
for rb in rbuttons:
    ifu, id = rb
    print("{} {} {}".format(ifu, id, rbuttons[rb].value))
    jj = (cat["ifu"] == ifu) * (cat["id"] == id)     
    if rbuttons[rb].value == "yes":
        cat["split_good"][jj] = True
    if rbuttons[rb].value == "no":
        cat["split_good"][jj] = False
    if rbuttons[rb].value == "unclear":
        cat["split_good"][jj] = None

In [ ]:
cat[ (cat["ifu"] == "023") * (cat["id"] == 5059) ]["split_good"]

In [ ]:
  
if not "xray_counterpart" in cat.colnames:
    cat.add_column(Column([None]*len(cat), name="xray_counterpart", description="XRay counterpart in civano+2016.", dtype="bool"))

print("XRay detect")  
for cbx in cboxes_xray:
    ifu, id = cbx
    print("{} {} {}".format(ifu, id, cboxes_xray[cbx].value))
    jj = (cat["ifu"] == ifu) * (cat["id"] == id)    
    cat["xray_counterpart"][jj] = cboxes_xray[cbx].value

In [ ]:
cat.write("../data/mmsf2outcube_COSMOSC_allifu.cat", format="ascii.ecsv")

In [ ]:
sum( cat["split_good"] == "True" )